In [11]:
import folium
import pandas as pd
from Distance import DistanceMatrix
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [12]:
df_locations = pd.read_csv('./Data/15loc.csv')
df_locations

,name,lat,lon
0,Denver_Colorado,39.7392,-104.9903
1,Key_West_Florida,24.5550,-81.8243
2,Chicago_Illinois,41.8781,-87.6298
3,Seattle_Washington,47.6062,-122.3321
4,New_York_City_New_York,40.7128,-74.0060
5,Los_Angeles_California,34.0522,-118.2437
6,Atlanta_Georgia,33.7489,-84.3879
7,Dallas_Texas,32.7767,-96.7970
8,Minneapolis_Minnesota,44.9778,-93.2650
9,Phoenix_Arizona,33.4484,-112.0740


In [13]:
dict_locations = df_locations.to_dict(orient='records')
distancematrix = DistanceMatrix(dict_locations)
distancematrix


[[0, 3567, 1921, 2133, 3405, 1736, 2532, 1386, 1461, 1225, 3610, 1266, 2053],
 [3567, 0, 2599, 5691, 2520, 4764, 1368, 2237, 3242, 4018, 275, 4455, 5620],
 [1921, 2599, 0, 3626, 1487, 3646, 1232, 1683, 742, 3038, 2493, 3183, 3671],
 [2133, 5691, 3626, 0, 5026, 2009, 4558, 3515, 2911, 2332, 5716, 1823, 303],
 [3405, 2520, 1487, 5026, 0, 5117, 1560, 2868, 2126, 4478, 2285, 4661, 5104],
 [1736, 4764, 3646, 2009, 5117, 0, 4044, 2590, 3183, 746, 4887, 478, 1729],
 [2532, 1368, 1232, 4558, 1560, 4044, 0, 1505, 1900, 3324, 1268, 3645, 4538],
 [1386, 2237, 1683, 3515, 2868, 2590, 1505, 0, 1807, 1851, 2323, 2237, 3414],
 [1461, 3242, 742, 2911, 2126, 3183, 1900, 1807, 0, 2675, 3168, 2706, 2978],
 [1225, 4018, 3038, 2332, 4478, 746, 3324, 1851, 2675, 0, 4141, 535, 2104],
 [3610, 275, 2493, 5716, 2285, 4887, 1268, 2323, 3168, 4141, 0, 4557, 5661],
 [1266, 4455, 3183, 1823, 4661, 478, 3645, 2237, 2706, 535, 4557, 0, 1582],
 [2053, 5620, 3671, 303, 5104, 1729, 4538, 3414, 2978, 2104, 5661, 1582, 0]

In [14]:
def create_data_model():
    data = {}
    data["distance_matrix"] = distancematrix
    data["num_vehicles"] = 2
    data["depot"] = 0
    return data

In [15]:
routes = []

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")

def get_routes(solution, routing, manager):
  global routes
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        1000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        # Display the routes.
        for i, route in enumerate(routes):
            print('Route', i, route) 
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()
    

Objective: 1032176
Route for vehicle 0:
 0 ->  8 ->  3 ->  12 ->  5 ->  11 ->  9 -> 0
Distance of the route: 8642m

Route for vehicle 1:
 0 ->  7 ->  1 ->  10 ->  6 ->  4 ->  2 -> 0
Distance of the route: 10134m

Maximum of the route distances: 10134m
Route 0 [0, 8, 3, 12, 5, 11, 9, 0]
Route 1 [0, 7, 1, 10, 6, 4, 2, 0]


In [16]:
print(routes)

[[0, 8, 3, 12, 5, 11, 9, 0], [0, 7, 1, 10, 6, 4, 2, 0]]


In [17]:
def mapping(df,routes):
    # Remove unused routes [0,0]
    filtered_routes = [route for route in routes if len(route) != 2]
    # Map coordinates for the remaining routes
    routes_coordinates = {}
    for i, route in enumerate(filtered_routes):
        coordinates = []
        for path in route:
            coordinates.append([df.iloc[path]['lat'], df.iloc[path]['lon']])
        routes_coordinates[i] = coordinates
    
    return routes_coordinates

coordinates = mapping(df_locations,routes)
coordinates

{0: [[np.float64(39.7392), np.float64(-104.9903)],
  [np.float64(44.9778), np.float64(-93.265)],
  [np.float64(47.6062), np.float64(-122.3321)],
  [np.float64(45.5234), np.float64(-122.6751)],
  [np.float64(34.0522), np.float64(-118.2437)],
  [np.float64(36.1699), np.float64(-115.1398)],
  [np.float64(33.4484), np.float64(-112.074)],
  [np.float64(39.7392), np.float64(-104.9903)]],
 1: [[np.float64(39.7392), np.float64(-104.9903)],
  [np.float64(32.7767), np.float64(-96.797)],
  [np.float64(24.555), np.float64(-81.8243)],
  [np.float64(25.7617), np.float64(-80.1918)],
  [np.float64(33.7489), np.float64(-84.3879)],
  [np.float64(40.7128), np.float64(-74.006)],
  [np.float64(41.8781), np.float64(-87.6298)],
  [np.float64(39.7392), np.float64(-104.9903)]]}

In [18]:
# Map
map_center = [df_locations['lat'].mean(), df_locations['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4, tiles=custom)

# Choose location
locations = df_locations

# Name of map
map_name = "./maps/VRP2.html"
map.save(map_name)

In [19]:
# Add marker
destination = df_locations.drop(index=0)

for _,row in destination.iterrows():
    folium.Circle(
        location=[row['lat'],row['lon']],
        radius=10000,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    radius=10000,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df_locations.iloc[0]['name']
).add_to(map)

map.save(map_name)

In [20]:
# Add edge
for coordinate in coordinates.values():
    folium.PolyLine(
        locations=coordinate,
        color="#ff6f00",
        weight=1,
        tooltip="TSP"
    ).add_to(map)

map.save(map_name)